In [155]:
import json

with open('notes.json') as f:
    data = json.load(f)

data = data['440']

def get_note(i, j):
    return fmt_note(data[i+j]['note'])

def fmt_note(note):
    return note.replace('#', '_sh_').upper()

bufer = ""
# notes
for element in data:
    bufer += f"pub const {fmt_note(element['note'])}: f32 = {float(element['frequency'])}" + ";\n"
    
bufer += "\n"

# minor scales
minor_scales = {}
stages = [0, 2, 3, 5, 7, 8, 10]
for i in range(0, (len(data)-24)):
    minor_scale = [get_note(i, j) for j in stages]
    minor_scales[get_note(i, 0) + 'M'] = minor_scale
    bufer += f"pub static {get_note(i, 0)}_MINOR_SCALE: [f32; 7] = [{', '.join(minor_scale)}];\n"

bufer += "\n"

# major scales
major_scales = {}
stages = [0, 2, 4, 5, 7, 9, 11]
for i in range(0, (len(data)-24)):
    major_scale = [get_note(i, j) for j in stages]
    major_scales[get_note(i, 0)] = major_scale
    bufer += f"pub static {get_note(i, 0)}_MAJOR_SCALE: [f32; 7] = [{', '.join(major_scale)}];\n"

bufer += "\n"

# minor leads
minor_leads = ["A", "A_SH_", "B", "C", "C_SH_", "D", "D_SH_", "E", "F", "F_SH_", "G", "G_SH_"]
stages = [3, 4, 5, 6]
for el in minor_leads:
    lead = ", ".join([f"&{el}{s}_MINOR_SCALE" for s in stages])
    bufer += f"pub static {el}M_LEAD: [&[f32; 7]; 4] = [{lead}];\n"
    
bufer += "\n"

# major leads
major_leads = ["A", "A_SH_", "B", "C", "C_SH_", "D", "D_SH_", "E", "F", "F_SH_", "G", "G_SH_"]
stages = [3, 4, 5, 6]
for el in major_leads:
    lead = ", ".join([f"&{el}{s}_MAJOR_SCALE" for s in stages])
    bufer += f"pub static {el}_LEAD: [&[f32; 7]; 4] = [{lead}];\n"
    
bufer += "\n"


# minor chords
stages = [0, 3, 5]
for i in range(12, (len(data)-5*12)):
    minor_chord = ", ".join([get_note(i, j) for j in stages])
    bufer += f"pub static {get_note(i, 0)}_MINOR_CHORD: [f32; 3] = [{minor_chord}];\n"

bufer += "\n"

# major chords
stages = [0, 4, 5]
for i in range(12, (len(data)-5*12)):
    major_chord = ", ".join([get_note(i, j) for j in stages])
    bufer += f"pub static {get_note(i, 0)}_MAJOR_CHORD: [f32; 3] = [{major_chord}];\n"
    
bufer += "\n"

# deminished chords
stages = [0, 3, 6]
for i in range(12, (len(data)-5*12)):
    dem_chord = ", ".join([get_note(i, j) for j in stages])
    bufer += f"pub static {get_note(i, 0)}_DEM_CHORD: [f32; 3] = [{dem_chord}];\n"
    
bufer += "\n"

# minor chords pools
note_lits = ["A", "A_SH_", "B", "C", "C_SH_", "D", "D_SH_", "E", "F", "F_SH_", "G", "G_SH_"]
octaves = ["1", "2"]
minor_stages = [0, 3, 4]
major_stages = [2, 5, 6]
dem_stages = [1]
for b in note_lits:
    minor_chords = ", ".join([f"&{minor_scales[b + o + 'M'][j]}_MINOR_CHORD" for j in minor_stages for o in octaves])
    major_chords = ", ".join([f"&{minor_scales[b + o + 'M'][j]}_MAJOR_CHORD" for j in major_stages for o in octaves])
    dem_chords = ", ".join([f"&{minor_scales[b + o + 'M'][j]}_DEM_CHORD" for j in dem_stages for o in octaves])
    bufer += f"pub static {b}M_CHORD_POOL: [&[f32; 3]; 14] = [{minor_chords}, {major_chords}, {dem_chords}];\n"
    
# major chords pools
note_lits = ["A", "A_SH_", "B", "C", "C_SH_", "D", "D_SH_", "E", "F", "F_SH_", "G", "G_SH_"]
octaves = ["1", "2"]
minor_stages = [1, 2, 5]
major_stages = [0, 3, 4]
dem_stages = [6]
for b in note_lits:
    minor_chords = ", ".join([f"&{major_scales[b + o][j]}_MINOR_CHORD" for j in minor_stages for o in octaves])
    major_chords = ", ".join([f"&{major_scales[b + o][j]}_MAJOR_CHORD" for j in major_stages for o in octaves])
    dem_chords = ", ".join([f"&{major_scales[b + o][j]}_DEM_CHORD" for j in dem_stages for o in octaves])
    if len(b) > 1:
        b = b[:-1]
    bufer += f"pub static {b}_CHORD_POOL: [&[f32; 3]; 14] = [{minor_chords}, {major_chords}, {dem_chords}];\n"

# # for el in minor_basses:
# #     for i in range(12, (len(data)-6*12)):
# #         minor_chords = ", ".join([f"{get_note(i, j)}_MINOR_CHORD" for j in minor_stages])
# #         major_chords = ", ".join([f"{get_note(i, j)}_MAJOR_CHORD" for j in major_stages])
# #     bufer += f"static {get_note(i, 0)}_CHORD_POOL: [&[f32; 3]; 12]"
# for scale, scale_notes in scales.items():
#     if scale[1] in ["1", "2"] or (scale[1] == '_' and scale[5] in ["1", "2"]):
#         minor_chords = ", ".join([f"&{scale_notes[j]}_MINOR_CHORD" for j in minor_stages])
#         major_chords = ", ".join([f"&{scale_notes[j]}_MAJOR_CHORD" for j in major_stages])
#         bufer += f"static {scale}_CHORD_POOL: [&[f32; 3]; 6] = [{minor_chords}, {major_chords}];\n"
    
with open('/mnt/f/ITMO/Projects/wav_project/src/notes.rs', 'w') as d:
    d.write(bufer)

In [150]:
minor_scales

{'C0M': ['C0', 'D0', 'D_SH_0', 'F0', 'G0', 'G_SH_0', 'A_SH_0'],
 'C_SH_0M': ['C_SH_0', 'D_SH_0', 'E0', 'F_SH_0', 'G_SH_0', 'A0', 'B0'],
 'D0M': ['D0', 'E0', 'F0', 'G0', 'A0', 'A_SH_0', 'C1'],
 'D_SH_0M': ['D_SH_0', 'F0', 'F_SH_0', 'G_SH_0', 'A_SH_0', 'B0', 'C_SH_1'],
 'E0M': ['E0', 'F_SH_0', 'G0', 'A0', 'B0', 'C1', 'D1'],
 'F0M': ['F0', 'G0', 'G_SH_0', 'A_SH_0', 'C1', 'C_SH_1', 'D_SH_1'],
 'F_SH_0M': ['F_SH_0', 'G_SH_0', 'A0', 'B0', 'C_SH_1', 'D1', 'E1'],
 'G0M': ['G0', 'A0', 'A_SH_0', 'C1', 'D1', 'D_SH_1', 'F1'],
 'G_SH_0M': ['G_SH_0', 'A_SH_0', 'B0', 'C_SH_1', 'D_SH_1', 'E1', 'F_SH_1'],
 'A0M': ['A0', 'B0', 'C1', 'D1', 'E1', 'F1', 'G1'],
 'A_SH_0M': ['A_SH_0', 'C1', 'C_SH_1', 'D_SH_1', 'F1', 'F_SH_1', 'G_SH_1'],
 'B0M': ['B0', 'C_SH_1', 'D1', 'E1', 'F_SH_1', 'G1', 'A1'],
 'C1M': ['C1', 'D1', 'D_SH_1', 'F1', 'G1', 'G_SH_1', 'A_SH_1'],
 'C_SH_1M': ['C_SH_1', 'D_SH_1', 'E1', 'F_SH_1', 'G_SH_1', 'A1', 'B1'],
 'D1M': ['D1', 'E1', 'F1', 'G1', 'A1', 'A_SH_1', 'C2'],
 'D_SH_1M': ['D_SH_1', '